## Errors + Questions
Hey Greg,

Any help with this error would be appreciated. Thanks!

#### Preliminaries

In [33]:
# load some packages
import plotly.express as px
from time import time
import networkx as nx
import pandas as pd
import oatpy as oat
import numpy as np
import pickle

# config
NETWORK_FILE = 'network.pickle'

#### Distance Matrix
Create the network we calculate persistence on and get a distance matrix

This isn't a well threashholded network for actually understanding anything, but it replicates the error well.

In [34]:
# network
G = pickle.load(open(NETWORK_FILE, 'rb'))

# distance matrix
adj = nx.adjacency_matrix(G, weight='norm_year') # adjacency matrix
node_births = np.array(list(nx.get_node_attributes(G, 'norm_year').values())) # node orgin years
adj.setdiag(node_births)
adj = adj.sorted_indices() # this is needed on some computers, not others. Idk why, but including it for that sake of completness

#### Homology Calculation
Use the `adj` distance matrix to calculate homology.

In [35]:
start = time()

# setup the problem
factored = oat.rust.FactoredBoundaryMatrixVr( # two functions that do this, idk what the other one is
        dissimilarity_matrix=adj,
        homology_dimension_max=2
    )

# solve homology
homology = factored.homology( # solve homology
        return_cycle_representatives=True, # These need to be true to be able to make a barcode, makes the problem take ~30% longer (1:30ish)
        return_bounding_chains=True
    )

f'Homology calculation took {time() - start} secs'

'Homology calculation took 5.2148332595825195 secs'

#### Optimize Cycle
Example of one that works as (I think) it should.

In [36]:
## Representative 1D Cycle
# index of cycle to optimize in homology dataframe
i = 1070 # works and is fun and nice

# optimization problem
start = time()
optimal = factored.optimize_cycle(
        birth_simplex=homology['birth simplex'][i], 
        problem_type='preserve PH basis'
    )
print(f'Optimizaiton took {time() - start} secs')

# filter coefficients ~0
dirty_optimal = optimal.loc['optimal cycle', 'chain'] # round all the dumb and bad coefficicents (This is in all my code now)
clean_optimal = dirty_optimal[round(dirty_optimal['coefficient'].astype(float)) != 0]
print(f'Removing {len(dirty_optimal)-len(clean_optimal)}/{len(dirty_optimal)} degenerate simplexes, {len(clean_optimal)} simplicies left')

# get nodes
rep_cycle_nodes = list(np.array(G.nodes)[pd.to_numeric(clean_optimal['simplex'].explode().drop_duplicates()).tolist()]) # this is gross
# gets the strings that correspond to all unique nodes

rep_cycle_nodes

Optimizaiton took 0.004465818405151367 secs
Finished construcing L1 optimization program.
Constraint matrix has 52 nonzero entries.
Passing program to solver.


Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 55, num_constraints: 80, objective: 2.5445544554455446 }
Removing 0/5 degenerate simplexes, 5 simplicies left


['algebraic equation',
 'analytical solution',
 'boundary control',
 'matrix riccati equation',
 'distribute parameter systems']

#### The Error
Many indices, especially lower ones, we get this error. 646 is one that I know gets it, but you can try any between 223 and about 800 and theres a decent chance you'll see the same thing.

In [37]:
## Representative 1D Cycle
# index of cycle to optimize in homology dataframe
i = 646 # causes error

# optimization problem
start = time()
optimal = factored.optimize_cycle(
        birth_simplex=homology['birth simplex'][i], 
        problem_type='preserve PH basis'
    )
print(f'Optimizaiton took {time() - start} secs')

# filter coefficients ~0
dirty_optimal = optimal.loc['optimal cycle', 'chain'] # round all the dumb and bad coefficicents
clean_optimal = dirty_optimal[round(dirty_optimal['coefficient'].astype(float)) != 0]
print(f'Removing {len(dirty_optimal)-len(clean_optimal)}/{len(dirty_optimal)} degenerate simplexes, {len(clean_optimal)} simplicies left')

# get nodes
rep_cycle_nodes = list(np.array(G.nodes)[pd.to_numeric(clean_optimal['simplex'].explode().drop_duplicates()).tolist()])

rep_cycle_nodes


Finished construcing L1 optimization program.
Constraint matrix has 6291 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 1982, num_constraints: 2226, objective: 3.336633663366589 }


thread '<unnamed>' panicked at /Users/gavinengelstad/Documents/Documents - Gavin’s MacBook Pro/School/Summer '24/Research/oat_suite/oatt/src/utilities/iterators/general.rs:417:21:


| ERROR: An iterator placed inside a `RequireStrictAscentWithPanic` struct has returned two consecutive entries, (x,y) where x > y.
| NB: This message can also appear when using a reversed order operator, indicating a failure to strictly *descend*.
| This error message is generated by OAT.




PanicException: 

| ERROR: An iterator placed inside a `RequireStrictAscentWithPanic` struct has returned two consecutive entries, (x,y) where x > y.
| NB: This message can also appear when using a reversed order operator, indicating a failure to strictly *descend*.
| This error message is generated by OAT.



#### Bonus: Bounding Chain Coefficients
Most bounding chains have "degenerate" simplicies in them that have coefficients ~0.

All of these are very near -1, 0, or 1, with most being near 0.

In [ ]:
## Representative Bounding Chain
# index of cycle to optimize in homology dataframe
i = 800

# optimization problem
start = time()
optimal = factored.optimize_bounding_chain(
        birth_simplex=homology['birth simplex'][i], 
    )
print(f'Optimizaiton took {time() - start} secs')

# get coefficients
dirty_optimal = optimal.loc['optimal bounding chain', 'chain']
dirty_optimal['coefficient'] = dirty_optimal['coefficient'].astype(float)
fig = px.histogram(dirty_optimal, x='coefficient', nbins=100)
fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 9950 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 3176, num_constraints: 4388, objective: 21.910891089116898 }
max difference in boundaries: Some(Ratio { numer: 2051389630267262849, denom: 2051389630267260928 })
Optimizaiton took 2.057110071182251 secs
